# LangChain Basics

## Import Packages

In [1]:
!pip install reportlab

In [48]:
import os
from dotenv import load_dotenv

from fpdf import FPDF
from operator import itemgetter

from langchain_openai import OpenAI, OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

In [3]:
load_dotenv()

True

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 1. LLMChain, PromptTemplate

In [5]:
# 기본적인 LLM 호출
llm = OpenAI(model="gpt-3.5-turbo-instruct")
response = llm.invoke("LangChain이 뭐야?")
print(response)


LangChain은 언어 체인 혹은 언어 사슬로서, 한 언어에서 다른 언어로의 연결 과정을 나타내는 개념입니다. 이는 특정 언어를 배우기 위해서는 그 언어와 관련된 언어들을 함께 배워야 한다는 것을 의미합니다. 예를 들어, 한국어를 배우기 위해서는 한국어와 관련된 언어인 중국어나 일본어도 함께 배우는 것이 도움이 될 수 있습니다. 따라서 LangChain은 다양한 언어를 연결하여 학습하는 방법을 말합니다. 


In [6]:
# 프롬프트 템플릿 활용
prompt = PromptTemplate.from_template("나를 {adjective} 개발자로 만들어줘.")
print(prompt.format(adjective="창의적인"))

나를 창의적인 개발자로 만들어줘.


In [7]:
# Runnable Sequence
chain = prompt | llm
response = chain.invoke({"adjective": "실용적인"})
print(response)



그건 나의 능력 밖이지만, 나는 최대한 노력해서 실용적인 개발자로 성장할 수 있도록 도와줄 수 있을 것 같아요. 함께 공부하고 경험을 쌓아가면서 실용적인 스킬을 익히고, 문제를 해결하는 능력을 키우는 것이 중요해요. 그리고 실제 프로젝트를 진행하며 실무에서 필요한 기술과 도구들을 익히는 것도 도움이 될 거에요. 또한, 다양한 분야의 개발에 대한 지식을 넓히고, 문제 해결 능력을 기르는 것도 중요해요. 그리고 끊임없이 배우고 발전하며, 좋은 개발자가 되기 위해 노


## 2. Few-shot Prompting

### 기본 Few-shot Prompting

In [8]:
# 기본 Few-shot Prompt
examples = [
    {"input": "강아지", "output": "귀엽고 털이 많은 포유류"},
    {"input": "고양이", "output": "유연하고 독립적인 성격의 동물"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음은 동물에 대한 설명입니다.",
    suffix="Input: {animal}\nOutput:",
    input_variables=["animal"],
)

print(fewshot_prompt.format(animal="기린"))

다음은 동물에 대한 설명입니다.

Input: 강아지
Output: 귀엽고 털이 많은 포유류

Input: 고양이
Output: 유연하고 독립적인 성격의 동물

Input: 기린
Output:


In [9]:
# Runnable Sequence
chain = fewshot_prompt | llm
response = chain.invoke({"animal": "기린"})
print(response)

 긴 목과 다양한 무늬를 가진 동물


### ExampleSelector을 이용한 동적 Few-shot Prompting

- example들이 많을 때, 입력과 유사한 예시만 자동 선택하는 방법
- ExampleSelector: abstract base class (ABC)

In [10]:
examples = [
    {"animal": "강아지", "desc": "사람과 친숙하고 애교 많은 포유류"},
    {"animal": "고양이", "desc": "독립적이며 유연한 동물"},
    {"animal": "토끼", "desc": "귀가 길고 점프를 잘함"},
    {"animal": "호랑이", "desc": "맹수이자 한국의 상징 동물"},
    {"animal": "기린", "desc": "목이 길고 초식 동물"},
    {"animal": "판다", "desc": "대나무를 먹는 흑백 털 동물"},
]

example_prompt = PromptTemplate(
    input_variables=["animal", "desc"],
    template="Input: {animal}\nOutput: {desc}"
)

In [11]:
# Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
# Example Selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # 전체 예시
    embedding_model, # 임베딩 방식
    Chroma, # 벡터 스토어
    k=3, # 몇 개 고를지
)

In [13]:
# Few-shot Prompt
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector, # examples와 동시에 사용 불가능
    example_prompt=example_prompt,
    prefix="다음은 동물 설명 예시입니다.",
    suffix="Input: {animal}\nOutput:",
    input_variables=["animal"]
)

prompt_str = few_shot_prompt.format(animal="사자")
print(prompt_str)

다음은 동물 설명 예시입니다.

Input: 판다
Output: 대나무를 먹는 흑백 털 동물

Input: 토끼
Output: 귀가 길고 점프를 잘함

Input: 기린
Output: 목이 길고 초식 동물

Input: 사자
Output:


## 3. Document Load & Chunking

- 다양한 형태의 문서를 Document 객체로 load
- 문서가 너무 클 경우, LLM에 넘기기 좋게 작은 청크로 분할
- RAG의 사전 작업으로 Embdding, Retrieval로 이어짐

### Sample PDF 생성

In [14]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.units import inch

# 나눔고딕 폰트 등록
pdfmetrics.registerFont(TTFont('NanumGothic', '/Users/kangjiwon/Library/Fonts/NanumGothic-Regular.ttf'))

# PDF 문서 설정
doc = SimpleDocTemplate(
    "./data/sample.pdf",
    pagesize=letter,
    rightMargin=72,  # 1인치
    leftMargin=72,   # 1인치
    topMargin=72,    # 1인치
    bottomMargin=72  # 1인치
)

# 한글 스타일 정의
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Korean',
    fontName='NanumGothic',
    fontSize=12,
    leading=16,      # 줄 간격
    alignment=TA_JUSTIFY
))

# 내용 준비
content = []
with open("./data/sample.txt", "r", encoding="utf-8") as f:
    text = f.read()
    # 각 문단을 Paragraph 객체로 변환
    paragraphs = text.split('\n')
    for para in paragraphs:
        if para.strip():  # 빈 줄 제외
            p = Paragraph(para, styles['Korean'])
            content.append(p)

# PDF 생성
doc.build(content)
print("PDF 생성 완료")

PDF 생성 완료


### 문서 Load

In [15]:
# PDF 문서
loader = PyPDFLoader("./data/sample.pdf")
docs = loader.load()
print(docs[0].page_content[:100])

LangChain을 활용한 LLM 기반 시스템 개요
LangChain은 다양한 외부 데이터와의 통합, 프롬프트 템플릿 구성, 체인 조합, 그리고
메모리 활용 기능을 포함하여 LLM


In [16]:
# 웹 페이지 로딩
loader = WebBaseLoader("https://ko.wikipedia.org/wiki/파이썬")
docs = loader.load()
print(docs[0].page_content[1400:1500])

ລາວLietuviųLatviešuМакедонскиമലയാളംМонголमराठीBahasa Melayuမြန်မာဘာသာPlattdüütschनेपालीNederlandsNor


In [17]:
# 텍스트 문서 로딩
loader = TextLoader("./data/sample.txt", encoding="utf-8")
docs = loader.load()
print(docs[0].page_content[:100])

LangChain을 활용한 LLM 기반 시스템 개요

LangChain은 다양한 외부 데이터와의 통합, 프롬프트 템플릿 구성, 체인 조합, 그리고 메모리 활용 기능을 포함하여 LL


### 문서 Chunking

In [18]:
# RecursiveCharacterTextSplitter: 문단 -> 줄바꿈 -> 문장 -> 단어 -> 문자 순으로 Chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5
)

chunks = splitter.split_documents(docs)

print(f"{len(chunks)} chunks created")
for i, chunk in enumerate(chunks[:5]):
    print(f"Chunk {i+1}: {chunk.page_content}...")

35 chunks created
Chunk 1: LangChain을 활용한 LLM...
Chunk 2: LLM 기반 시스템 개요...
Chunk 3: LangChain은 다양한 외부...
Chunk 4: 외부 데이터와의 통합, 프롬프트...
Chunk 5: 프롬프트 템플릿 구성, 체인 조합,...


## 4. Embedding & Vector-store

### Embedding들을 Vectore-store에 저장

In [19]:
loader = PyPDFLoader("./data/sample.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5
)
chunks = splitter.split_documents(docs)

In [20]:
embedding_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./vector_db",
)

print(f"총 {vectorstore._collection.count()}개의 벡터가 저장됨")

총 108개의 벡터가 저장됨


### 유사한 문서 검색

In [21]:
query = "LLM 시스템의 핵심 구성요소는?"
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"[결과 {i+1}]: {doc.page_content}")

[결과 1]: LLM 기반 시스템 개요
[결과 2]: LLM 기반 시스템 개요
[결과 3]: LLM 기반 시스템 개요


### Vectorstore 저장 & 재사용

In [22]:
vectorstore = Chroma(
    persist_directory="./vector_db",
    embedding_function=embedding_model,
)

## 5. Retrieval + QA

### Retriever

In [32]:
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0)

In [31]:
# 프롬프트 템플릿 구성
template = """너는 친절한 LLM 도우미야. 다음 context를 참고해서 질문에 답변해줘.
Context:
{context}

Question:
{question}
"""

prompt = PromptTemplate.from_template(template)

In [37]:
# Retriever는 Document 객체의 리스트를 반환하지만 LLM은 문자열 입력을 기대
# 각 Document를 page_content로 추출하고 이를 결합하여 하나의 문자열로 만듬
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### LCEL 방식의 Retrieval

In [41]:
# LCEL (LangChain Expression Language) 사용
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

query = "LangChain의 핵심 구성 요소는 뭐야?"
response = rag_chain.invoke(query)
print(f"LLM 답변: {response.content}")

LLM 답변: LangChain의 핵심 구성 요소는 다양한 외부 데이터 소스와 연결되어 있는 언어 모델이다.


### 각 단계를 명시적으로 구현한 Retrieval

In [40]:
# 실제 작동 방식
docs = retriever.invoke(query) # [Document1, Document2, ...]
context = format_docs(docs) # "Document1의 내용\n\nDocument2의 내용\n\n..."

inputs = {
    "context": context,
    "question": query
}

formatted_prompt = prompt.format(**inputs)
response = llm.invoke(formatted_prompt)
print(f"LLM 답변: {response.content}")

LLM 답변: LangChain의 핵심 구성 요소는 다양한 외부 데이터 소스와 연결되어 있는 언어 모델이다.


### 출처 문서 반환

RunnableMap
- 병렬 실행, 스트리밍, 비동기 실행 등 지원
- 에러 핸들링 내장

In [50]:
# LLM 초기화
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

# 벡터 DB 생성 또는 로드
embedding_model = OpenAIEmbeddings()
vectorstore = Chroma(
    persist_directory="./vector_db",
    embedding_function=embedding_model,
)

# retriever 생성
retriever = vectorstore.as_retriever()

- PromptTemplate: 단순 텍스트 출력
- ChatPromptTemplate: 메세지 역할 (system, user, assistant) 구분 가능

In [51]:
prompt = ChatPromptTemplate.from_template("""다음 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트: {context}

질문: {question}""")

In [52]:
query = {"question": "LangChain의 핵심 구성 요소는 뭐야?", "language": "ko"}

# 1단계: 문서 검색
context_chain = itemgetter("question") | retriever # retriever에 질문 텍스트만 전달

# 2단계: 답변 생성
rag_chain = {
    "context": context_chain | format_docs,
    "question": itemgetter("question") # 프롬프트 템플릿에 원본 질문 제공
} | prompt | llm

# 3단계: 전체 결과 구성
full_chain = RunnableMap({
    "result": rag_chain,
    "source_documents": context_chain
})

In [53]:
output = full_chain.invoke(query)
for doc in output["source_documents"]:
    print(doc.metadata["source"]) # 문서 출처
    print(doc.page_content) # 문서 내용

./data/sample.pdf
LangChain을 활용한 LLM
./data/sample.pdf
LangChain을 활용한 LLM
./data/sample.pdf
LangChain을 활용한 LLM
./data/sample.pdf
LangChain은 다양한 외부
